In [1]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python
retention_utils.py:448: error: Name 'get_ipython' is not defined
efficacy_utils.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils.py:163: error: Name 'get_ipython' is not defined
efficacy_utils.py:180: error: Name 'get_ipython' is not defined
efficacy_utils.py:184: error: Name 'get_ipython' is not defined
efficacy_utils.py:188: error: Name 'get_ipython' is not defined


In [2]:
from retention_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  #condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  condition_to_installs = get_conditions_to_install_list_in_abtest(abtest_name, groups)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [8]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [9]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [10]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    get_epoch_to_domain_to_time_spent(install)
    #print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

In [13]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
#         total_time_spent = sum(domain_to_time_spent.values())
#         output.append({
#           'user': install,
#           'epoch': epoch,
#           'time': total_time_spent,
#           'logtime': math.log(total_time_spent),
#           'condition': condition,
#         })
        for domain,total_time_spent in  domain_to_time_spent.items():
          output.append({
            'domain': domain,
            'user': install,
            'epoch': epoch,
            'time': total_time_spent,
            'logtime': math.log(total_time_spent),
            'condition': condition,
          })
  return to_dataframe(output)

In [32]:
# noexport

df = make_domain_to_daily_time_dataframe()

In [15]:
#print(df)

In [34]:
# noexport

%Rpush df

/usr/local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning:

Error while trying to convert the column "condition". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'



In [17]:
#%%R
#install.library

In [35]:
%%R

library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)


In [19]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [36]:
%%R

df$user <- as.factor(df$user)
df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard", "survey", "nodefault_forcedchoice_userchoice"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)




[[1]]
[1] "www.google.com  : 150738  "

[[2]]
[1] "www.youtube.com : 105926  "

[[3]]
[1] "mail.google.com :  95790  "

[[4]]
[1] "www.facebook.com:  71314  "

[[5]]
[1] "docs.google.com :  58150  "

[[6]]
[1] "drive.google.com:  39631  "

[[7]]
[1] "(Other)         :4089948  "

[[8]]
[1] "f9329b2116e3bd031cca0519:  19134  "

[[9]]
[1] "ac3fb867e496b79bd34dff4a:  17419  "

[[10]]
[1] "1e42870bb128918b18137b18:  16560  "

[[11]]
[1] "1e02d5363a5d3f7ae81fbbc1:  14910  "

[[12]]
[1] "0063f0e0e7858c3c37a1586a:  14144  "

[[13]]
[1] "c58636d9d2fd7f7d6640f3e1:  14115  "

[[14]]
[1] "(Other)                 :4515215  "

[[15]]
[1] "1578   :  25779  "

[[16]]
[1] "1567   :  25361  "

[[17]]
[1] "1573   :  25224  "

[[18]]
[1] "1566   :  25218  "

[[19]]
[1] "1551   :  25142  "

[[20]]
[1] "1550   :  25113  "

[[21]]
[1] "(Other):4459660  "

[[22]]
[1] "Min.   :    1.0  "

[[23]]
[1] "1st Qu.:   14.0  "

[[24]]
[1] "Median :   55.0  "

[[25]]
[1] "Mean   :  372.9  "

[[26]]
[1] "3rd Qu.:  200.0

In [21]:
%%R

#dfsub <- df[df$domain == 'www.youtube.com', ]
#show(summary(dfsub))
#show(summary(subset(df, domain='www.youtube.com')))
#dfsub <- subset(df, domain='www.youtube.com')
#show(summary(dfsub))

NULL


In [37]:
%%R

dfsub <- df
show(summary(dfsub))
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = dfsub)
results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

              domain                              user        
 www.google.com  : 150738   f9329b2116e3bd031cca0519:  19134  
 www.youtube.com : 105926   ac3fb867e496b79bd34dff4a:  17419  
 mail.google.com :  95790   1e42870bb128918b18137b18:  16560  
 www.facebook.com:  71314   1e02d5363a5d3f7ae81fbbc1:  14910  
 docs.google.com :  58150   0063f0e0e7858c3c37a1586a:  14144  
 drive.google.com:  39631   c58636d9d2fd7f7d6640f3e1:  14115  
 (Other)         :4089948   (Other)                 :4515215  
     epoch              time            logtime      
 1578   :  25779   Min.   :    1.0   Min.   : 0.000  
 1567   :  25361   1st Qu.:   14.0   1st Qu.: 2.639  
 1573   :  25224   Median :   55.0   Median : 4.007  
 1566   :  25218   Mean   :  372.9   Mean   : 4.008  
 1551   :  25142   3rd Qu.:  200.0   3rd Qu.: 5.298  
 1550   :  25113   Max.   :89087.0   Max.   :11.397  
 (Other):4459660                                     
                             condition     
 survey_nochoice_not

In [22]:
%%R

dfsub <- df[df$domain == 'www.facebook.com', ]
show(summary(dfsub))
#results <- lmer(logtime ~ condition + (1|user), data = df)
results <- lmer(logtime ~ condition + (1|user), data = dfsub)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

                   domain                            user           epoch      
 www.facebook.com     :71314   3d8f23613f3849ed266ace58:  338   1573   :  388  
 _.shoprintee.com     :    0   f9329b2116e3bd031cca0519:  278   1557   :  386  
 _antonia.gr          :    0   79f9aa9fe47ba2f0fa59ba73:  270   1565   :  381  
 _dmarc.adoreme.com   :    0   71da006ec218a9b8e3eb317e:  265   1574   :  378  
 _dnslink.your.website:    0   26c1461ecb3d3dd841ecbaac:  263   1566   :  377  
 _kunden.suedcom.info :    0   e02d077981cded46b3146e43:  257   1587   :  376  
 (Other)              :    0   (Other)                 :69643   (Other):69028  
      time          logtime                                   condition    
 Min.   :    1   Min.   : 0.000   survey_nochoice_nothing          : 9994  
 1st Qu.:  114   1st Qu.: 4.736   survey_nochoice_easy             :12776  
 Median :  445   Median : 6.098   survey_nochoice_medium           :12055  
 Mean   : 1128   Mean   : 5.813   survey_nochoice_hard  

In [23]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [24]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [25]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [26]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [27]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey 1096.5792787635946
nodefault_forcedchoice_userchoice 942.6471468827052
survey_nochoice_hard 1207.3468691878488
survey_nochoice_nothing 990.8107864718831
survey_nochoice_easy 1330.2546180338134
survey_nochoice_medium 1148.6735794276233
Ttest_indResult(statistic=-4.19997517189966, pvalue=2.6784397884211723e-05)


In [28]:
# noexport

from plot_utils import *


In [29]:
# noexport

#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [30]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break